In [1]:
import pandas as pd
import random

In [2]:
# Requirements Generation
reqs = [
    # Functional - Monitoring
    ["MD-FUNC-001", "The app SHALL display real-time Heart Rate (BPM) data received from the sensor within 1 second of transmission.", "Continuous Patient Monitoring"],
    ["MD-FUNC-002", "The app SHALL display Oxygen Saturation (SpO2) percentage data with an accuracy of +/- 2% as reported by the sensor.", "Continuous Patient Monitoring"],
    ["MD-FUNC-003", "The app SHALL update the Respiratory Rate (RPM) visualization every 5 seconds based on incoming telemetry.", "Continuous Patient Monitoring"],
    ["MD-FUNC-004", "The app SHALL provide a 'Patient Dashboard' view summarizing all active vital signs for the assigned clinician.", "Clinician Workflow Efficiency"],
    ["MD-FUNC-005", "The app SHALL allow clinicians to manually enter Blood Pressure readings (Systolic/Diastolic) when a manual cuff is used.", "Data Integrity"],
    
    # Safety & Alerting (IEC 62304 / ISO 14971)
    ["MD-SAFE-001", "The app SHALL trigger a High Priority Alert (Audible > 70dB and flashing red Visual) within 2 seconds if Heart Rate exceeds the High Threshold.", "Patient Safety & Rapid Response"],
    ["MD-SAFE-002", "The app SHALL trigger a Medium Priority Alert (Audible and yellow Visual) if the sensor battery level falls below 15%.", "Risk Management"],
    ["MD-SAFE-003", "The app SHALL notify the clinician via a 'Loss of Signal' critical alert if data is not received from the sensor for > 10 seconds.", "Patient Safety & Connectivity"],
    ["MD-SAFE-004", "The app SHALL require an explicit clinician 'Acknowledge' action to silence High Priority Alerts.", "Clinical Accountability"],
    ["MD-SAFE-005", "The app SHALL log all threshold breaches and clinician acknowledgments with a millisecond-accurate timestamp to the audit trail.", "Regulatory Compliance (IEC 62304)"],
    
    # Performance
    ["MD-PERF-001", "The app SHALL maintain a UI frame rate of at least 30 FPS during simultaneous monitoring of up to 4 vital sign waveforms.", "System Reliability"],
    ["MD-PERF-002", "Alert delivery from the cloud gateway to the mobile device notification center SHALL occur in < 500ms under 4G/5G/Wi-Fi conditions.", "Critical Alert Latency"],
    ["MD-PERF-003", "The app SHALL support background operation, ensuring alerts are delivered even when the device is locked or the app is not in focus.", "Continuous Monitoring Reliability"],
    
    # Security & Privacy (HIPAA/GDPR)
    ["MD-SEC-001", "The app SHALL encrypt all Patient Identifiable Information (PII) and Protected Health Information (PHI) at rest using AES-256.", "Data Privacy & HIPAA Compliance"],
    ["MD-SEC-002", "The app SHALL require Multi-Factor Authentication (MFA) for clinician login using Biometrics or Hardware Security Keys.", "Access Control"],
    ["MD-SEC-003", "The app SHALL automatically lock the session after 2 minutes of clinician inactivity on the patient monitoring screen.", "Security Compliance"],
    ["MD-SEC-004", "All telemetry data transmitted between the sensor and the app SHALL be encrypted via TLS 1.3.", "Cybersecurity (ISO 21434)"],
    
    # Usability & Connectivity
    ["MD-USER-001", "The app SHALL follow WCAG 2.1 Level AA standards for color contrast to ensure readability in low-light hospital environments.", "Usability"],
    ["MD-USER-002", "The app SHALL provide a Bluetooth LE pairing wizard with signal strength indicators to assist clinicians in sensor placement.", "Ease of Use"],
    ["MD-USER-003", "The app SHALL allow clinicians to toggle between Dark Mode and Light Mode based on clinical environment needs.", "Usability"],
    
    # Integration/System
    ["MD-INT-001", "The app SHALL synchronize patient vital sign history with the Hospital Electronic Health Record (EHR) via HL7 FHIR R4 API.", "Interoperability"],
    ["MD-INT-002", "The app SHALL support 'Ward Mode' allowing a single tablet to display status for up to 10 patients simultaneously.", "Scalability"],
    ["MD-INT-003", "The app SHALL function in 'Offline Mode' by caching up to 24 hours of data if the local Wi-Fi connection is lost.", "Fault Tolerance"]
]

requirements_df = pd.DataFrame(reqs, columns=['requirement_id', 'requirement', 'high_level_goals'])

In [3]:
# Test Case Generation
test_cases = []
traces = []

req_to_tc_map = {
    "MD-FUNC-001": [
        ("TC-MD-001", "Verify real-time HR display updates within 1s.", "Sensor simulator connected; Heart rate set to 75 BPM.", "1. Send HR packet from simulator.\n2. Observe app UI timestamp.", "1. App UI shows 75 BPM.\n2. Delay between packet and display is < 1s."),
        ("TC-MD-002", "Verify HR display handles erratic sensor data.", "Sensor simulator connected.", "1. Send 0 BPM.\n2. Send 300 BPM (invalid high).\n3. Send NULL.", "1. App displays '---' for 0 or invalid.\n2. App triggers technical alert for data anomaly.")
    ],
    "MD-SAFE-001": [
        ("TC-MD-003", "Verify High Priority Alert on HR threshold breach.", "Patient HR threshold set to 120 BPM; HR currently 80 BPM.", "1. Increase simulator HR to 125 BPM.\n2. Measure time to alert.", "1. Flashing Red UI triggered.\n2. High-pitch audible alarm starts.\n3. Latency < 2s."),
        ("TC-MD-004", "Verify High Priority Alert volume meets 70dB spec.", "Sound meter calibrated; App in monitoring mode.", "1. Trigger alert.\n2. Measure decibels at 1 meter.", "1. Sound level is >= 70dB.")
    ],
    "MD-SAFE-003": [
        ("TC-MD-005", "Verify Loss of Signal alert triggers after 10s.", "Active sensor connection.", "1. Power off sensor.\n2. Observe timer.", "1. At T+10s, 'Loss of Signal' critical alert appears.\n2. Haptic feedback triggered.")
    ],
    "MD-SEC-001": [
        ("TC-MD-006", "Verify PHI encryption at rest.", "Rooted/Jailbroken test device with app data.", "1. Capture app SQLite database.\n2. Attempt to read patient_name field.", "1. Data is obfuscated/encrypted.\n2. Hex editor shows AES-256 pattern, no plain text.")
    ],
    "MD-PERF-003": [
        ("TC-MD-007", "Verify background alert delivery.", "App running in background; Device locked.", "1. Trigger High HR alert from simulator.", "1. System notification appears on lock screen.\n2. Critical Alert bypasses Do Not Disturb (if configured).")
    ],
    "MD-INT-001": [
        ("TC-MD-008", "Verify FHIR R4 API synchronization.", "Sandbox EHR environment connected.", "1. Complete monitoring session.\n2. Query EHR Observation endpoint.", "1. Vital signs present in EHR.\n2. Correct patient UUID and timestamp recorded.")
    ]
}

# Expand test cases to ensure 100% coverage and realistic scale
all_tcs = []
tc_id_counter = 100

for i, row in requirements_df.iterrows():
    r_id = row['requirement_id']
    r_text = row['requirement']
    
    # If not already defined in map, create 2 standard TCs (Positive & Negative/Boundary)
    if r_id not in req_to_tc_map:
        tc_id_pos = f"TC-MD-{tc_id_counter}"
        tc_id_neg = f"TC-MD-{tc_id_counter + 1}"
        tc_id_counter += 2
        
        # Positive TC
        tc_pos = (tc_id_pos, f"To verify that {r_text[:50]}...", "System in normal operating state.", "1. Perform standard operation.\n2. Verify outcome.", "1. System behaves as specified in requirement.")
        # Negative/Boundary TC
        tc_neg = (tc_id_neg, f"Verify {r_id} resilience under error conditions.", "System in restricted state.", "1. Provide invalid input or interrupt process.\n2. Observe error handling.", "1. System handles error gracefully without crashing.")
        
        req_to_tc_map[r_id] = [tc_pos, tc_neg]

# Now flatten req_to_tc_map into the steps dataframe format
steps_data = []
traces_data = []

req_id_to_num = {row['requirement_id']: 9100000 + i for i, row in requirements_df.iterrows()}
tc_id_to_num = {}
tc_num_counter = 9200001

for r_id, tcs in req_to_tc_map.items():
    from_item_num = req_id_to_num[r_id]
    for tc in tcs:
        t_id, desc, setup, steps, expected = tc
        
        if t_id not in tc_id_to_num:
            tc_id_to_num[t_id] = tc_num_counter
            tc_num_counter += 1
            
            # Create full testcase string
            tc_str = f"Test Case ID: {t_id}\nTest Case Objective: {desc}\nPrerequisites: {setup}\nSteps:\n{steps}\n\nExpectedResults:\n{expected}"
            
            steps_data.append([t_id, desc, setup, steps, expected, tc_str])
        
        to_item_num = tc_id_to_num[t_id]
        traces_data.append([from_item_num, r_id, to_item_num, t_id])

steps_df = pd.DataFrame(steps_data, columns=['documentKey', 'description', 'setup', 'all_steps', 'all_expectedResults', 'testcases'])
traces_df = pd.DataFrame(traces_data, columns=['fromItem', 'fromItem_documentKey', 'toItem', 'toItem_documentKey'])

# Save to Excel
with pd.ExcelWriter('MedicalApp_Traceability_Dataset.xlsx', engine='xlsxwriter') as writer:
    requirements_df.to_excel(writer, sheet_name='Requirements', index=False)
    steps_df.to_excel(writer, sheet_name='Test_Cases', index=False)
    traces_df.to_excel(writer, sheet_name='Traceability_Matrix', index=False)
    
    # Formatting
    workbook = writer.book
    header_format = workbook.add_format({'bold': True, 'bg_color': '#D3D3D3', 'border': 1})
    wrap_format = workbook.add_format({'text_wrap': True, 'valign': 'top'})
    
    for sheet in writer.sheets.values():
        sheet.set_column('A:Z', 30, wrap_format)
        # sheet.set_row(0, None, header_format) # Note: can't easily apply to header only via this api easily, but default header is okay.
        # Let's try to format headers explicitly
        
    # Apply header format to first row
    for sheet_name in writer.sheets:
        worksheet = writer.sheets[sheet_name]
        df = requirements_df if sheet_name == 'Requirements' else (steps_df if sheet_name == 'Test_Cases' else traces_df)
        for col_num, value in enumerate(df.columns.values):
            worksheet.write(0, col_num, value, header_format)
        worksheet.freeze_panes(1, 0)

print("Excel file created successfully.")

Excel file created successfully.
